In [1]:
!pwd
# !ls

/home/dephinate/ASU/DL/MedicalChatBot/research


In [2]:
import os
from pathlib import Path

In [3]:
os.chdir("../")

In [4]:
!pwd

/home/dephinate/ASU/DL/MedicalChatBot


In [5]:
# # GPU llama-cpp-python
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip -I install llama-cpp-python numpy --force-reinstall --upgrade --no-cache-dir --verbose
# !pip install huggingface_hub
# !pip install -I llama-cpp-python
# !pip install numpy
# !pip install accelerate

In [6]:
# !pip install ctransformers
# !pip install sentence-transformers
# !pip install pinecone-client
# !pip install langchain
# !pip install langchain_pinecone
# !pip install flask
# !pip install pypdf
# !pip install ctransformers[cuda]

In [7]:
# Print distribution
import numpy as np
def print_stats(len_list,docs):
    len_arr = np.array(len_list)
    print("num of chunks: ",len(len_arr))
    print("min: ",np.argmin(len_arr) ,np.min(len_arr))
    print("max: ",np.argmax(len_arr) ,np.max(len_arr))
    print("avg :",np.mean(len_arr))
    print("std :",np.std(len_arr))

In [8]:
# Formatting
import re
from unstructured.cleaners.core import clean,group_broken_paragraphs
def format_docs(doc):
    print(doc.page_content)
    para_split_re = re.compile(r"(\s*\n\s*){3}")
    print("\nRestructured: ",group_broken_paragraphs(doc.page_content,paragraph_split=para_split_re))
    print("\nSource: ",doc.metadata)

In [9]:
import re
from unstructured.cleaners.core import clean,group_broken_paragraphs
para_split_re = re.compile(r"(\s*\n\s*){3}")

Create Embeddings

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings
# fucntion to download hugging face embeddings
def download_embeddings_from_huggingface(model_name:str):
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = download_embeddings_from_huggingface(model_name=embedding_model)

/home/dephinate/miniconda3/envs/medicalChatBot_recent/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load Data

In [33]:
import pypdf
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

loader = DirectoryLoader(   # To load all pdfs from a directory
    path="artifacts/data",
    glob="*.pdf",
    loader_cls=PyPDFLoader,
    show_progress=True
)
documents = loader.load()

100%|██████████| 1/1 [00:14<00:00, 14.27s/it]


In [43]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# splitter = RecursiveCharacterTextSplitter(chunk_size = 500  , chunk_overlap = 20, separators=['\n\n', '\n', '.'])
splitter = RecursiveCharacterTextSplitter(chunk_size = 500  , chunk_overlap = 20)
chunks = splitter.split_documents(documents)

Pinecone

In [23]:
# Initialize index
from pinecone import Pinecone
pc = Pinecone(
    api_key = "ce80aa7c-c98e-467c-a100-b4b7e6a07c05",
)
pc.list_indexes().names() # just to doublecheck if I am able to connect to my Index
index_name = "medical-chatbot"
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [24]:
# Create Vectorstore
from langchain_pinecone import PineconeVectorStore
# docs_chunks =[t.page_content for t in chunks]

vectorstore = PineconeVectorStore(
    index=index,
    embedding=embeddings,
    namespace="medicalChatBot",
    index_name='medical-chatbot'
)

In [42]:
# # Testing cleanup and check status
# vectorizer.clean_pinecone_db(db_instance=pc)
pc.Index(index_name).delete(delete_all=True,namespace="medicalChatBot")
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [44]:
# vectorstore.add_texts(texts=[group_broken_paragraphs(t.page_content,paragraph_split=para_split_re) for t in chunks])
vectorstore.add_documents(chunks)

['46db64af-d4a3-4266-bf6e-e0b36868e794',
 '433436ca-08be-4af8-a3db-d2cfe1e048dc',
 '1007cff6-edb5-4610-9f33-d43d0d52bfab',
 '6d09d953-8dc5-4103-9a78-448f1502e51f',
 '89bd5085-843f-47b7-b112-88b1a8d07e82',
 'd821ba92-3c2c-4eba-a179-c962e9f66868',
 '19b4964b-2e68-4354-864e-19658e594474',
 'bff259af-b99f-48a1-9a30-afb14980cbaa',
 'f806a1cd-1505-428a-b7a1-a09ce3c33c18',
 '59f3159d-b2a4-421e-b08b-cdff66263223',
 '21a47b8d-48cb-4d74-8705-2c866a34c428',
 '5ef1acf0-0e3c-4919-b565-120dcd5535f9',
 '66972af7-d5c7-4553-9c75-062c5fb0fef5',
 'd91f981d-3e28-4c33-a410-6777852fa718',
 '4f2fbe51-0cd6-48f3-93d3-a5e5d765ffde',
 '9dcecfa8-6671-4a26-9804-d782f69c086a',
 'aa14aa81-dfc2-46c0-bb70-a01423623fb3',
 'd4837b35-40e0-4a60-968f-44072eb71d1f',
 '1ef22014-4eff-4d49-8631-0ba2e9008de1',
 '70ccef38-e561-4219-94bf-1be99be1fe80',
 '61632f05-7cc1-4c21-ac56-020c01ce2b7d',
 '2412a1e7-fb93-4b3a-8783-3a50a192fa96',
 '24c07c9a-ee2f-4be2-9efe-c4d0e4c4adad',
 '446cdaa2-fe97-487f-9548-f8948a4b8b72',
 '7aa8d734-d324-

In [45]:

query = "What is Acne?"

chunks_retrieved = vectorstore.similarity_search_with_relevance_scores(
    query,  # our search query
    k=5,  # return 3 most relevant docs
    # fetch_k = 30
)
i = 0
for chunk in chunks_retrieved:
    # print(i,":",group_broken_paragraphs(chunk[0].page_content,paragraph_split=para_split_re),"\n")
    print(i,":",chunk[0].page_content,chunk[0].metadata,"\n")
    # print(i,":",chunk,"\n")

    

    i = i+1

0 : Acidosis seeRespiratory acidosis; Renal
tubular acidosis; Metabolic acidosis
Acne
Definition
Acne is a common skin disease characterized by
pimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.
Description
Acne vulgaris, the medical term for common acne, is {'page': 37.0, 'source': 'artifacts/data/Medical_book.pdf'} 

1 : GALE ENCYCLOPEDIA OF MEDICINE 2 25Acne
Acne vulgaris affecting a woman’s face. Acne is the general
name given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-
ates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25 {'page': 38.0, 'source': 'artifacts/data/Medical_book.pdf'} 

2 : ent purposes. For example, lotions, soaps, gels, andcreams containing benzoyl peroxide or tretinoin may beused to clear up mild to moderately severe acne.Isotretinoin (Accutane) is prescribed only for verysevere, disfi

Using LlamaCpp

In [46]:
from langchain_community.llms import LlamaCpp

In [47]:
# GPU
lcpp_llm = None
lcpp_llm = LlamaCpp(
    model_path="artifacts/model/llama-2-7b-chat.Q5_K_M.gguf",
    n_gpu_layers=32,
    n_batch=512,
    n_ctx=1024,
    f16_kv=True, 
    temperature = 0
    )

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from artifacts/model/llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32

In [48]:
# Template String
template4="""
<s>[INST] <<SYS>>
Answer the question on the information. If you don't know the answer, only return 'I don't know', Do not make an answer.
<</SYS>>
question: {question}
information:{context}[/INST]

"""

In [63]:
template5="""
[INST] <<SYS>>
Based on the information, I will answer the question as a medical assistant, If you don't know the answer I will return 'I don't know'. I will not make an answer.
<</SYS>>
{question}

information:{context}[/INST]

"""

In [56]:
default_template="""
[INST] <<SYS>>
Answer the query based on the information. If you don't know the answer, return 'I do not know', Do not make an answer.
<</SYS>>
query:{question}
information:{context}[/INST]

"""

In [65]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template(template5)
prompt

PromptTemplate(input_variables=['context', 'question'], template="\n[INST] <<SYS>>\nBased on the information, I will answer the question as a medical assistant, If you don't know the answer I will return 'I don't know'. I will not make an answer.\n<</SYS>>\n{question}\n\ninformation:{context}[/INST]\n\n")

In [66]:
from langchain.chains import RetrievalQA
qa=RetrievalQA.from_chain_type(
    llm=lcpp_llm, 
    chain_type="stuff", 
    retriever=vectorstore.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True, 
    chain_type_kwargs={"prompt": prompt},
    verbose = True)

In [70]:
query = "How to improve sleep?"

response = qa.invoke({"query":query})
# lcpp_llm.invoke(query)



> Entering new RetrievalQA chain...


Llama.generate: prefix-match hit

llama_print_timings:        load time =    2104.98 ms
llama_print_timings:      sample time =      51.98 ms /   256 runs   (    0.20 ms per token,  4925.26 tokens per second)
llama_print_timings: prompt eval time =   23265.44 ms /   326 tokens (   71.37 ms per token,    14.01 tokens per second)
llama_print_timings:        eval time =   97421.53 ms /   255 runs   (  382.05 ms per token,     2.62 tokens per second)
llama_print_timings:       total time =  121499.41 ms /   581 tokens



> Finished chain.


In [72]:
response['source_documents']


[Document(page_content='Melatonin may be helpful for insomnia . It has also\nbeen used to help fight viruses and bacterial infections,reduce the risk of heart disease, improve sexual function-ing, and to protect against cancer.\nHuman growth hormone (hGH) has been shown to\nregulate blood sugar levels and to stimulate bone, carti-lage, and muscle growth while reducing fat.\nHerbs\nGarlic ( Allium sativa ) is helpful in preventing heart', metadata={'page': 84.0, 'source': 'artifacts/data/Medical_book.pdf'}),
 Document(page_content='Patients who are taking benzodiazepines for sleep\nproblems should check with their physicians if they arenot sleeping better within 7-10 days. Sleep problems thatlast longer than this may be a sign of another medicalproblem.\nPeople who take this medicine to help them sleep\nmay have trouble sleeping when they stop taking themedicine. This effect should last only a few nights.\nSome people, especially older people, feel drowsy,', metadata={'page': 479.0, 'so

In [71]:
response['result']

'Based on the information provided, here are some ways to improve sleep:\n\n1. Melatonin: Melatonin may be helpful for insomnia and has been used to fight viruses and bacterial infections, reduce the risk of heart disease, improve sexual functioning, and protect against cancer. However, it is important to consult with a physician before taking melatonin supplements.\n2. Human growth hormone (hGH): hGH has been shown to regulate blood sugar levels and stimulate bone, cartilage, and muscle growth while reducing fat. However, it is important to consult with a physician before taking hGH supplements.\n3. Garlic (Allium sativa): Garlic may be helpful in preventing heart problems. However, it is important to consult with a physician before taking garlic supplements.\n4. Benzodiazepines: Patients who are taking benzodiazepines for sleep problems should check with their physicians if they are not sleeping better within 7-10 days. Sleep problems that last longer than this may be a sign of anoth

M2

In [41]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template4)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="\n[INST] <<SYS>>\nAnswer the question based on the context.If you don't know the answer, only return I don't know, Do not make an answer.\n<</SYS>>\nquestion:{question}\ncontext:{context}[/INST]\n\n"))])

In [42]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema.output_parser import StrOutputParser

In [43]:
retrieval = RunnableParallel(
    {"context": vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2}) | format_docs, "question": RunnablePassthrough()}
)

chain = retrieval | prompt | lcpp_llm | StrOutputParser()

In [44]:
retrieval.invoke("Caffien?")

{'context': '1. The caffeine acts by constricting blood vessels to relievethe headache . Sometimes, an analgesic such as aceta-\nminophen , caffeine, and a barbiturate which acts as a\n2. • reducing or stopping consumption of caffeine, alcohol,\nor tobacco; and/or\n• discontinuing diet pills or other medications (over-the-\ncounter or prescription)\nResources\nBOOKS\nCurrent Medical Diagnosis and Treatment, 1998. 37th ed. Ed.\nStephen McPhee, et al. Stamford: Appleton & Lange,1997.\nDeBakey, Michael E., and Antonio M. Gotto Jr. The New Liv-',
 'question': 'Caffien?'}

In [56]:
(retrieval|prompt).invoke("what is Acne and its diffent types and treatment?")

StringPromptValue(text="\n[INST] <<SYS>>\nAnswer the question based on the context.If you don't know the answer, only return I don't know, Do not make an answer.\n<</SYS>>\nquestion:what is Acne and its diffent types and treatment?\ncontext:1. Acne treatment consists of reducing sebum produc-\ntion, removing dead skin cells, and killing bacteria withtopical drugs and oral medications. Treatment choicedepends upon whether the acne is mild, moderate, orsevere.\nDrugs\nTOPICAL DRUGS. Treatment for mild noninflamma-\n2. Acne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is[/INST]\n\n")

In [57]:

result = chain.invoke("what is Acne and its diffent types and treatment?")
result

Llama.generate: prefix-match hit

llama_print_timings:        load time =   17519.10 ms
llama_print_timings:      sample time =     110.15 ms /   256 runs   (    0.43 ms per token,  2324.06 tokens per second)
llama_print_timings: prompt eval time =   16386.90 ms /   227 tokens (   72.19 ms per token,    13.85 tokens per second)
llama_print_timings:        eval time =  109442.39 ms /   255 runs   (  429.19 ms per token,     2.33 tokens per second)
llama_print_timings:       total time =  126737.09 ms /   482 tokens


'Based on the provided context, here are the different types of acne and their treatments:\nTypes of Acne:\n1. Mild acne: This type of acne is characterized by blackheads and whiteheads. Treatment for mild acne includes topical drugs such as benzoyl peroxide, salicylic acid, and sulfur.\n2. Moderate acne: This type of acne is characterized by papules, pustules, and nodules. Treatment for moderate acne includes topical drugs such as retinoids, oral antibiotics, and hormonal treatments.\n3. Severe acne: This type of acne is characterized by cysts and abscesses. Treatment for severe acne includes oral antibiotics, hormonal treatments, and Isotretinoin (a powerful oral medication).\nTopical Drugs:\nTopical drugs are applied directly to the skin to reduce sebum production, remove dead skin cells, and kill bacteria that cause acne. Some common topical drugs used to treat acne include:\n'

In [47]:
print(result)

Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.


In [58]:
lcpp_llm.invoke("what is Acne and its diffent types and treatment?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =   17519.10 ms
llama_print_timings:      sample time =     110.62 ms /   256 runs   (    0.43 ms per token,  2314.19 tokens per second)
llama_print_timings: prompt eval time =    2328.33 ms /    12 tokens (  194.03 ms per token,     5.15 tokens per second)
llama_print_timings:        eval time =   84516.55 ms /   255 runs   (  331.44 ms per token,     3.02 tokens per second)
llama_print_timings:       total time =   87661.71 ms /   267 tokens


'\nWhat is Acne?Acne is a common skin condition that occurs when the pores on the skin become clogged with dead skin cells, oil, and bacteria. It can cause whiteheads, blackheads, and inflamed red pimples on the face, as well as other areas of the body. Acne can lead to emotional distress, social anxiety, and low self-esteem.\nTypes of acne:\nThere are several different types of acne, including:\n1. Comedonal acne: This type of acne occurs when dead skin cells and excess oil clog the pores, causing whiteheads and blackheads.\n2. Inflammatory acne: This type of acne occurs when a clogged pore becomes infected, leading to redness, swelling, and pus-filled pimples.\n3. Cystic acne: This is the most severe form of acne, causing large, painful cysts that can lead to scarring.\nTreatment for acne:\nTreatment for acne depends on the severity of the condition and may include:\n1.'

In [32]:
print(result)

3. Treatments for acne include over-the-counter (OTC)
medications and prescription medications. OTC treat-ments include benzoyl peroxide, salicylic acid, and sulfur. Prescription treatments include antibiotics, retinoids, and hormone therapy.
4. Avoid picking or squeezing acne as it can lead to infection and
scarring.
5. Maintain good hygiene habits such as washing your face twice daily with a mild soap and lukewarm water, and using non-comedogenic products to help prevent acne.
6. Eating a healthy diet rich in fruits, vegetables, whole grains, and lean protein can help to reduce acne.
7. Stress can trigger or worsen acne, so finding ways to manage stress such as exercise, meditation or deep breathing can be helpful.
8. Getting enough sleep can also help to reduce acne as lack of sleep can lead to increased stress levels which can contribute to acne.
9. Avoid touching your face as it can transfer bacteria
